In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import pickle
import xgboost as xgb

In [18]:
#use pickle file
SPLIT_POINT = 590540


with open('train_test_590540.pkl', 'rb') as f:
    df_all_dy = pickle.load(f)
    print("df_all load finished")

    
#df_train = df_all_dy[:SPLIT_POINT]
#df_train = df_train.drop(['index'], axis=1)

df_test = df_all_dy[SPLIT_POINT:].reset_index()
df_test = df_test.drop(['isFraud', 'level_0', 'index'], axis=1)

#del df_all_dy 

df_all load finished


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 266 entries, C1 to card6__nan
dtypes: float16(187), float32(22), float64(1), int16(1), int32(2), uint8(53)
memory usage: 300.2 MB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 265 entries, C1 to card6__nan
dtypes: float16(187), float32(22), int16(1), int32(2), uint8(53)
memory usage: 253.7 MB


In [9]:
#param['gpu_id'] = 1 # 从GPU 1 开始

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    #missing=-999,
    random_state=2019,
    tree_method='gpu_hist',  # THE MAGICAL PARAMETER
    #gpu_id = 1
)

%time clf.fit(df_train.drop('isFraud', axis=1), df_train['isFraud'])

Wall time: 11min 57s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=2019,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [10]:
with open('xgb-win-1.pkl', 'wb') as f:
    pickle.dump(clf, f)
    print("clf dump finished")
    
'''
with open('df_merged_train.pkl', 'wb') as f:
    pickle.dump(df_train, f)
    print("df_train dump finished")
'''

clf dump finished


'\nwith open(\'df_merged_train.pkl\', \'wb\') as f:\n    pickle.dump(df_train, f)\n    print("df_train dump finished")\n'

In [11]:
gc.collectectectect()

144

In [16]:
with open('xgb-win-1.pkl', 'rb') as f:
    clf = pickle.load(f)
    print("clf load finished")
    

clf load finished


In [25]:

sample_submission = pd.read_csv('data/ieee-fraud-detection/sample_submission.csv')

#sample_submission['isFraud'] = clf.predict_proba(df_test)[:,1]
batch = 1000
for i in range((sample_submission.shape[0]+batch-1) // batch + 1):
    if st >= sample_submission.shape[0]:
        break
    st = (i-1) * batch if i > 0 else 0
    ed = i * batch
    ed = ed if ed <= sample_submission.shape[0] else sample_submission.shape[0]
    
    tmp = clf.predict_proba(df_test.iloc[st:ed])[:, 1]
    if i % 100 == 0:
        print(st, ed)
    #print(sample_submission.iloc[st:ed, 1], tmp.shape)
    sample_submission.iloc[st:ed, 1] = clf.predict_proba(df_test.iloc[st:ed])[:, 1]


sample_submission.to_csv('simple_xgboost-2.csv', index=False)

0 0
99000 100000
199000 200000
299000 300000
399000 400000
499000 500000


In [4]:
ceshi = pd.read_csv('data/ieee-fraud-detection/sample_submission.csv')


In [24]:
gc.collect()

138

In [26]:
sample_submission

,TransactionID,isFraud
0,3663549,0.002358
1,3663550,0.000738
2,3663551,0.000941
3,3663552,0.001490
4,3663553,0.000671
5,3663554,0.003121
6,3663555,0.009387
7,3663556,0.021463
8,3663557,0.000165
9,3663558,0.007579


In [ ]:
df_test.isnull().sum().sum()

In [ ]:
df_train['isFraud']

In [28]:
df_test['TransactionID']

0         3663549
1         3663550
2         3663551
3         3663552
4         3663553
5         3663554
6         3663555
7         3663556
8         3663557
9         3663558
10        3663559
11        3663560
12        3663561
13        3663562
14        3663563
15        3663564
16        3663565
17        3663566
18        3663567
19        3663568
20        3663569
21        3663570
22        3663571
23        3663572
24        3663573
25        3663574
26        3663575
27        3663576
28        3663577
29        3663578
           ...   
506661    4170210
506662    4170211
506663    4170212
506664    4170213
506665    4170214
506666    4170215
506667    4170216
506668    4170217
506669    4170218
506670    4170219
506671    4170220
506672    4170221
506673    4170222
506674    4170223
506675    4170224
506676    4170225
506677    4170226
506678    4170227
506679    4170228
506680    4170229
506681    4170230
506682    4170231
506683    4170232
506684    4170233
506685    